In [ ]:
# import packages
import pandas as pd
import numpy as np
import openpyxl as pxl

# load data
data = pd.read_json ('/content/supplier_car.json', lines=True)

In [ ]:
pip install XlsxWriter

1) Step Pre-processing

In [ ]:
# sort data for pre-processing
sorted_data = data.sort_values(['ID', 'Attribute Names'])
s_d = sorted_data
s_d = s_d.drop(labels = ['entity_id', 'ModelTypeText', 'TypeNameFull', 'ModelText'], axis = 1)
s_d = s_d.rename(columns={"MakeText" : "make", 'TypeName': 'model'})

In [ ]:
# add a row for the Harley-Davidson
HDrow = {'ID': 824, 'make': 'HARLEY-DAVIDSON', 'model': 'HPU Hurricane TC', 'Attribute Names': 'BodyTypeText', 'Attribute Values': 'Motocycle'}
s_d = s_d.append(HDrow, ignore_index = True)

In [ ]:
# create the colnames variable that contains column names
colnames = s_d['Attribute Names'].unique()
colnames = colnames.tolist()
colnames.insert(0,'model')
colnames.insert(0,'make')
colnames.insert(0,'ID')

In [ ]:
# pre-process data and input these data in a new dataframe
# new dataframe creation
new_dataframe = pd.DataFrame(data = None, columns = colnames)
n_d = new_dataframe
# creation of the colnames_2 variable that equals colnames except 3 columns names
colnames_2 = colnames
colnames_2.remove('ID')
colnames_2.remove('make')
colnames_2.remove('model')
# index creation
for i in s_d['ID'].unique() :
  index = s_d['ID'] == i
  indexed_data = s_d[index]
  i_d = indexed_data
  # first dataframe line
  if n_d.shape[0] == 0 :
   # first 3 columns
   n_d['ID'] = i_d['ID'].unique()
   n_d['make'] = i_d['make'].unique()
   n_d['model'] = i_d['model'].unique()
   # remaining columns
   for j in colnames_2 :
     i = i
     jndex = i_d['Attribute Names'] == j
     jndexed_data = i_d[jndex]
     j_d = jndexed_data
     n_d[j].iloc[i-1] = j_d['Attribute Values'].unique()
  # following dataframe lines
  else  :
   # first 3 columns
   n_d['ID'].iloc[-1] = i_d['ID'].unique()
   n_d['make'].iloc[-1] = i_d['make'].unique()
   n_d['model'].iloc[-1] = i_d['model'].unique()
   # remaining columns
   for j in colnames_2 :
     i = i
     jndex = i_d['Attribute Names'] == j
     jndexed_data = i_d[jndex]
     j_d = jndexed_data
     n_d[j].iloc[i-1] = j_d['Attribute Values'].unique()
  n_d = n_d.append(new_dataframe, ignore_index = True) 
n_d = n_d.drop(axis = 0, index = 1153)
#n_d

In [ ]:
# removing brackets from strings within the dataframe
n_d = n_d.astype(str)
x = 1153
y = 22
arr = np.zeros((x, y), dtype=int)
colnames.insert(0,'model')
colnames.insert(0,'make')
colnames.insert(0,'ID')
n_d_2 = pd.DataFrame(data = arr, columns = colnames)
for k in colnames :
  n_d_2[k] = n_d[k].str.strip("[']")
#n_d_2

2) Step Normalization

In [ ]:
# import Target data
df = pd.read_excel ('/content/Target Data.xlsx')

In [ ]:
# visualize conditions in pre-processed data
n_d_2['ConditionTypeText'].unique()

In [ ]:
# visualize conditions in target data
df['condition'].unique()

In [90]:
# data normalisation for new & used, other labeling have to be discussed with the customer
n_d_3 = n_d_2.replace(to_replace=['Occasion', 'Neu'], value=['Used', 'New'], inplace=False)
n_d_3['ConditionTypeText'].unique()

array(['Used', 'Oldtimer', 'New', 'Vorführmodell'], dtype=object)

3) Step Integration

In [ ]:
# rename the BodyTypeText column 
n_d_3_2 = n_d_3.rename(columns = {'BodyTypeText':'carType'}, inplace = False)
#n_d_3_2.head()

In [ ]:
# set the carType column from the normalized data as the first column
n_d_3_2 = n_d_3_2[['carType'] + [l for l in n_d_3_2 if l not in ['carType']]]
#n_d_3_2.head()

In [110]:
# join the target and normalized dataframes
n_d_4 = pd.concat([df, n_d_3_2])
n_d_4 = n_d_4.reset_index(drop=True)

In [111]:
# export datasheets to an excel file
with pd.ExcelWriter('/content/Deliverable Hugo JOHAN.xlsx', engine='xlsxwriter') as writer:
   n_d_2.to_excel(writer, sheet_name='Pre-processed data', index=False)
   n_d_3.to_excel(writer, sheet_name='Normalized supplier data', index=False)
   n_d_4.to_excel(writer, sheet_name='Integrated supplier data', index=False)